### Ingestion de la carpeta "production_company"


In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
productions_companies_schema = StructType(fields=[
     StructField("companyId", IntegerType(), True),
     StructField("companyName", StringType(), True),
])

In [0]:
# El comodin de production_company_*.csv, es para indicar que cargue todos los archivos que comiencen con
# production_company. Pero para este caso todos los archivos tienen como nombre inicial production_company_01.csv, production_company_02.csv, etc. Entonces no hace falta agreagar el comoding _*.csv. Solo lo dejamos hasta la carpeta
productions_companies_df = spark.read \
                   .schema(productions_companies_schema) \
                   .csv(f"{bronze_folder_path}/{v_file_date}/production_company") 
                   #.csv("/mnt/sacjccmoviehistory/bronze/production_company/production_company_*.csv")

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "companyId" renombrar a "company_id"
2. "companyName" renombrar a "company_name"
3. Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_companies_final_df = (
    add_ingestion_date(productions_companies_df)
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date)) \
    .withColumnsRenamed({"companyId": "company_id", "companyName": "company_name"})
)

### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(productions_companies_final_df,"movie_silver","productions_companies","file_date")

In [0]:
# productions_companies_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_companies")

# Crear una tabla en base al data frame productions_companies_final_df
# Creara la tabla productions_companies en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver

#productions_companies_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.productions_companies")

# Append, agrega los registros a la tabla existente, carga incremental de datos

#productions_companies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_companies")

#Modulo: Delta Lake
merge_condition = "tgt.company_id = src.company_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(productions_companies_final_df,"movie_silver","productions_companies",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.productions_companies
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/productions_companies"))
display(spark.read.format("delta").load(f"{silver_folder_path}/productions_companies")) 

In [0]:
dbutils.notebook.exit("Success")